In [47]:
import pandas as pd
from pathlib import Path

base = Path("/Users/oreo/Desktop/My Folder/UC Berkeley/Projects/coWeight")

# Input files
f_cowdb = base / "data" / "labels" / "Manual_measurements.xlsx"
f_cowdb2 = base / "data" / "labels" / "Database2022v2.xlsx"

# Output CSVs
out_cowdb  = base / "data" / "metadata" / "labels_cowdb.csv"
out_cowdb2 = base / "data" / "metadata" / "labels_cowdatabase2.csv"

# -------------------------------
# CowDB (manual_measurements.xlsx)
# -------------------------------
df_cowdb = pd.read_excel(f_cowdb, sheet_name="result")

# Keep only the needed columns, rename
df_cowdb = df_cowdb[["N", "live weithg"]].rename(columns={
    "N": "cow_id",
    "live weithg": "weight_kg"
})

# Clean up cow_id (remove leading zeros, force string)
df_cowdb["cow_id"] = df_cowdb["cow_id"].astype(str).str.strip().str.lstrip("0")
df_cowdb["dataset"] = "CowDB"

# Save
df_cowdb.to_csv(out_cowdb, index=False)
print(f"[OK] Saved {len(df_cowdb)} rows -> {out_cowdb}")

# -----------------------------------
# CowDatabase2 (Database2022v2.xlsx)
# -----------------------------------
df_cowdb2 = pd.read_excel(f_cowdb2, sheet_name="Лист1")

# Choose the identifier that matches your folder names.
# Try Tag number first; if not matching, switch to Chip number or №
df_cowdb2 = df_cowdb2[["Tag number", "Live weigth"]].rename(columns={
    "Tag number": "cow_id",
    "Live weigth": "weight_kg"
})

# Clean up cow_id
df_cowdb2["cow_id"] = df_cowdb2["cow_id"].astype(str).str.strip().str.lstrip("0")
df_cowdb2["dataset"] = "CowDatabase2"

# Save
df_cowdb2.to_csv(out_cowdb2, index=False)
print(f"[OK] Saved {len(df_cowdb2)} rows -> {out_cowdb2}")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/oreo/Desktop/My Folder/UC Berkeley/Projects/coWeight/data/labels/Manual_measurements.xlsx'

In [91]:
df = pd.read_excel(f_cowdb2)
df

,N,Frame number,Live weigth,Withers height,Hip height,Chest width,Chest heart
0,1,3,633,123,122,53,214
1,2,4,632,119,124,52,224
2,3,5,604,120,122,50,217
3,4,7,617,145,144,51,216
4,5,8,619,143,143,56,215
...,...,...,...,...,...,...,...
91,92,108,581,143,145,51,217
92,93,109,582,142,147,53,217
93,94,110,605,149,146,51,216
94,95,111,592,142,143,54,222


In [101]:
from pathlib import Path
import pandas as pd, numpy as np, re, shutil

# === CONFIG ===
BASE = Path("/Users/oreo/Desktop/My Folder/UC Berkeley/Projects/coWeight")
RAW_DB2 = BASE / "data" / "raw" / "CowDatabase2"
LABELS_XLSX = BASE / "data" / "labels" / "Database2022v2.xlsx"  # your cleaned Excel
SHEET_NAME = None  # None => first sheet

OUT_DIR = BASE / "data" / "metadata"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_FRAMES = OUT_DIR / "db2_frames_manifest.csv"
OUT_ALIGNED = OUT_DIR / "db2_labels_aligned.csv"

# === helpers ===
def clean_id(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace(" ","")
    if re.fullmatch(r"\d+", s): return str(int(s))
    s = re.sub(r"[^\d]", "", s)
    return str(int(s)) if s else np.nan

def detect_column(cols, candidates):
    lowmap = {str(c).strip().lower(): c for c in cols}
    for cand in candidates:
        k = cand.lower()
        if k in lowmap:
            return lowmap[k]
    # loose contains
    for cand in candidates:
        k = cand.lower().replace("_","").replace(" ","")
        for lc, orig in lowmap.items():
            if k in lc.replace("_","").replace(" ",""):
                return orig
    return None

def load_labels_detect():
    xls = pd.ExcelFile(LABELS_XLSX)
    sheet = SHEET_NAME or xls.sheet_names[0]
    df = xls.parse(sheet)
    # detect columns
    cowid_col = detect_column(
        df.columns,
        ["cow_id","№","N","Number","Tag number","Chip number","ID","Id","Cow"]
    )
    frame_col = detect_column(
        df.columns,
        ["frame_number","Frame number","Frame","frame","fn","FN"]
    )
    weight_col = detect_column(
        df.columns,
        ["weight_kg","Live weigth","Live weithg","Live weight","Weight","weight"]
    )

    print("Detected columns:",
          "\n  cow_id     ->", cowid_col,
          "\n  frame_num  ->", frame_col,
          "\n  weight_kg  ->", weight_col)
    if cowid_col is None or frame_col is None:
        print("\nAvailable columns in Excel:", list(df.columns))
        raise ValueError("Could not detect cow_id or frame_number column. Update candidate lists above.")

    out = df[[cowid_col, frame_col] + ([weight_col] if weight_col else [])].copy()
    out.rename(columns={cowid_col:"cow_id", frame_col:"frame_number", (weight_col or "weight_kg"):"weight_kg"}, inplace=True, errors="ignore")
    out["cow_id"] = out["cow_id"].apply(clean_id)
    out["frame_number"] = pd.to_numeric(out["frame_number"], errors="coerce").astype("Int64")
    if "weight_kg" in out.columns:
        out["weight_kg"] = pd.to_numeric(out["weight_kg"], errors="coerce")
    out = out.dropna(subset=["cow_id","frame_number"])
    out["frame_number"] = out["frame_number"].astype(int)
    # group set of frames per cow
    keep_map = {cid: set(g["frame_number"].tolist()) for cid, g in out.groupby("cow_id")}
    return out, keep_map

FRAME_PATTERNS = [
    r"frame[_\- ]?(\d+)",
    r"\bf[_\- ]?(\d+)\b",
    r"\bfn[_\- ]?(\d+)\b",
    r"(\d+)"  # last resort
]

def parse_frame_from_name(name: str):
    nm = name.lower()
    for pat in FRAME_PATTERNS:
        m = re.findall(pat, nm)
        if m:
            s = re.sub(r"[^\d]","", m[-1])
            if s.isdigit():
                return int(s)
    return None

def list_view_files(view_dir: Path):
    return sorted([f for f in view_dir.glob("*.*")
                   if f.is_file() and f.suffix.lower() in [".png",".jpg",".jpeg",".tif",".tiff",".ply"]])

def scan_files():
    rows = []
    for cow_dir in sorted(RAW_DB2.iterdir()):
        if not cow_dir.is_dir(): continue
        cow_id = cow_dir.name
        raw_dirs = [d for d in cow_dir.iterdir() if d.is_dir() and "raw" in d.name.lower()]
        if not raw_dirs: continue
        raw_dir = raw_dirs[0]
        for view in ["left","right","top"]:
            vdir = next((d for d in raw_dir.iterdir() if d.is_dir() and d.name.lower()==view), None)
            if vdir is None: continue
            files = list_view_files(vdir)
            for f in files:
                rows.append({
                    "cow_id": cow_id,
                    "view": view,
                    "file_path": str(f),
                    "file_name": f.name,
                    "frame_number_parsed": parse_frame_from_name(f.name)
                })
    return pd.DataFrame(rows)

def align_by_order(files_df, keep_frames_set):
    """Align by sorted order when filenames carry no usable numbers."""
    files_sorted = files_df.sort_values("file_name").reset_index(drop=True)
    frames_sorted = sorted(list(keep_frames_set))
    n = min(len(files_sorted), len(frames_sorted))
    aligned = files_sorted.iloc[:n].copy()
    aligned["frame_number"] = frames_sorted[:n]
    leftover_files = files_sorted.iloc[n:]
    leftover_frames = frames_sorted[n:]
    return aligned, leftover_files, leftover_frames

# ---- RUN ----
labels_df, keep_map = load_labels_detect()
files_df = scan_files()

# Save full frames manifest (with parsed numbers if any)
manifest = files_df.copy()
manifest.to_csv(OUT_FRAMES, index=False)

# For each cow+view, prefer filename parsing; if none parsed, align by order
kept_rows = []
quarantine_rows = []
for (cow_id, view), g in files_df.groupby(["cow_id","view"], sort=False):
    want = keep_map.get(clean_id(cow_id), set())
    if not want:
        # no frames requested for this cow -> quarantine all files for transparency
        quarantine_rows.append(g)
        continue

    # Use filename parse if present
    parsed = g.dropna(subset=["frame_number_parsed"]).copy()
    matched = parsed[parsed["frame_number_parsed"].isin(want)].copy()
    if len(matched) > 0:
        matched["frame_number"] = matched["frame_number_parsed"]
        kept_rows.append(matched)
        # remaining files in this group that aren't matched = quarantine
        quarantine_rows.append(g.loc[~g["file_path"].isin(matched["file_path"])])
    else:
        # fallback: align by order
        aligned, leftovers_files, leftovers_frames = align_by_order(g, want)
        kept_rows.append(aligned)
        quarantine_rows.append(leftovers_files)

kept = pd.concat(kept_rows, ignore_index=True) if kept_rows else pd.DataFrame(columns=list(files_df.columns) + ["frame_number"])
quarantine = pd.concat(quarantine_rows, ignore_index=True) if quarantine_rows else pd.DataFrame(columns=files_df.columns)

# attach weight if present in labels
if "weight_kg" in labels_df.columns:
    kept = kept.merge(labels_df[["cow_id","frame_number","weight_kg"]], on=["cow_id","frame_number"], how="left")

# Save aligned (kept) pairs
kept.to_csv(OUT_ALIGNED, index=False)

print("=== Summary ===")
print("Total files scanned:", len(files_df))
print("Labels rows:", len(labels_df))
print("Kept (aligned) rows:", len(kept))
print("Quarantine candidates:", len(quarantine))
print(f"\nWrote manifest -> {OUT_FRAMES}")
print(f"Wrote aligned  -> {OUT_ALIGNED}")

# Peek unmatched cases
print("\nExamples of files with no match (first 10):")
display(quarantine.head(10))

# Peek kept
print("\nExamples of kept matches (first 10):")
display(kept.head(10))


Detected columns: 
  cow_id     -> N 
  frame_num  -> Frame number  
  weight_kg  -> Live weigth
=== Summary ===
Total files scanned: 1071
Labels rows: 96
Kept (aligned) rows: 282
Quarantine candidates: 789

Wrote manifest -> /Users/oreo/Desktop/My Folder/UC Berkeley/Projects/coWeight/data/metadata/db2_frames_manifest.csv
Wrote aligned  -> /Users/oreo/Desktop/My Folder/UC Berkeley/Projects/coWeight/data/metadata/db2_labels_aligned.csv

Examples of files with no match (first 10):


,cow_id,view,file_path,file_name,frame_number_parsed
0,1,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,depth.png,None
1,1,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,rgb.png,None
2,1,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,depth.png,None
3,1,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,rgb.png,None
4,1,top,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,depth.png,None
5,1,top,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,rgb.png,None
6,100,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None
7,100,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,depth.png,None
8,100,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,rgb.png,None
9,100,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None



Examples of kept matches (first 10):


,cow_id,view,file_path,file_name,frame_number_parsed,frame_number,weight_kg
0,1,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,3,633
1,1,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,3,633
2,1,top,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,3,633
3,12,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,17,626
4,12,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,17,626
5,12,top,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,17,626
6,13,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,18,625
7,13,right,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,18,625
8,13,top,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,18,625
9,14,left,/Users/oreo/Desktop/My Folder/UC Berkeley/Proj...,cloud.ply,None,19,600
